- 시계열 데이터에 대한 예측
    - HEPC(house hold electric power consumption) dataset : 다변량 시계열 데이터
- tensorflow로 모델 개발
- torch로 동일한 모델 개발

# HEPC dataset
모든 컬럼에 대한 forecasting을 수행

In [2]:
import numpy as np
import pandas as pd
import urllib
import zipfile

url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
urllib.request.urlretrieve(url, 'household_power.zip')
with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
    zip_ref.extractall()
hepc = pd.read_csv('household_power_consumption.csv')
print(hepc.shape)
hepc.head()

(86400, 8)


,datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [3]:
# normalize
data_origin = hepc.values[:,1:]
data = data_origin.astype('float32')
data = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))
n_features = data.shape[1]


In [4]:
# train, valid split
train_size = int(len(data) * 0.5)
train_data, valid_data = data[:train_size, :], data[train_size:, :]

# tensorflow

In [5]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size= (n_past + n_future),shift = shift, drop_remainder = True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(
        lambda w: (w[:n_past], w[n_past:])
    )
    return ds.batch(batch_size).prefetch(1)

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Bidirectional

# 0~23의 데이터를 이용해 뒤의 24~47의 데이터를 예측함
n_past = 24
n_future = 24
batch_size = 32
# window_size = n_past + n_future

tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

train_set = windowed_dataset(train_data, batch_size, n_past, n_future)
valid_set = windowed_dataset(valid_data, batch_size, n_past, n_future)

In [7]:
for i,(x_tf,y_tf) in enumerate(train_set.take(1)):
    if i==1:
        break

In [8]:
x_tf.shape, y_tf.shape

(TensorShape([32, 24, 7]), TensorShape([32, 24, 7]))

In [9]:
# first batch
x_tf[0,:10,0], y_tf[0,:10,0]

(<tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([0.44304916, 0.56906813, 0.5706103 , 0.5721525 , 0.38246307,
        0.36638024, 0.3864287 , 0.38620842, 0.3826834 , 0.38202247],
       dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([0.47146946, 0.3364177 , 0.33509585, 0.33421457, 0.33751926,
        0.32870677, 0.27825513, 0.39259747, 0.45692882, 0.4756554 ],
       dtype=float32)>)

In [10]:
train_data[:10,0], train_data[n_past:n_past+10,0]

(array([0.44304916, 0.56906813, 0.5706103 , 0.5721525 , 0.38246307,
        0.36638024, 0.3864287 , 0.38620842, 0.3826834 , 0.38202247],
       dtype=float32),
 array([0.47146946, 0.3364177 , 0.33509585, 0.33421457, 0.33751926,
        0.32870677, 0.27825513, 0.39259747, 0.45692882, 0.4756554 ],
       dtype=float32))

In [11]:
def equal_two_tensor(a,b):
    aa,_ = tf.unique(tf.reshape(a==b,-1))
    return aa.numpy()
equal_two_tensor(y_tf[0,:,:],train_data[n_past:n_past+n_future, :])

array([ True])

In [12]:
# 참고
print(x_tf.shape)
x_conv_tf = keras.layers.Conv1D(filters=32, kernel_size=3, padding='causal', activation='relu', input_shape=[n_past, n_features])(x_tf)
print(x_conv_tf.shape)
# Bidrectional은 양방향으로 학습할 수 있게 함
x_lstm1_tf = Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x_conv_tf)
print(x_lstm1_tf.shape)
x_lstm2_tf = Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x_lstm1_tf)
print(x_lstm2_tf.shape)
x_dense1_tf = keras.layers.Dense(32, activation='relu')(x_lstm2_tf)
print(x_dense1_tf.shape)

# (32,24,128)은 batch size : 32, sequence_length (n_past) : 24, feature의 개수 64*2 를 의미함

(32, 24, 7)
(32, 24, 32)
(32, 24, 128)
(32, 24, 128)
(32, 24, 32)


In [13]:
model = keras.models.Sequential([
    # causal padding은 시계열 데이터에서 미래의 데이터를 미리 보는 일을 방지하기 위해 과거 데이터쪽으로만 padding을 추가
    keras.layers.Conv1D(filters=32, kernel_size=3, padding='causal', activation='relu', input_shape=[n_past, n_features]),
    Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(n_features)
])

optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

# callback은 생략
model.fit(train_set, epochs=20, validation_data=valid_set)

Epoch 1/20
1349/1349 [==============================] - 34s 19ms/step - loss: 0.0968 - mae: 0.0968 - val_loss: 0.0720 - val_mae: 0.0720
Epoch 2/20
1349/1349 [==============================] - 24s 18ms/step - loss: 0.0704 - mae: 0.0704 - val_loss: 0.0653 - val_mae: 0.0653
Epoch 3/20
1349/1349 [==============================] - 23s 17ms/step - loss: 0.0633 - mae: 0.0633 - val_loss: 0.0637 - val_mae: 0.0637
Epoch 4/20
1349/1349 [==============================] - 23s 17ms/step - loss: 0.0608 - mae: 0.0608 - val_loss: 0.0625 - val_mae: 0.0625
Epoch 5/20
1349/1349 [==============================] - 23s 17ms/step - loss: 0.0592 - mae: 0.0592 - val_loss: 0.0588 - val_mae: 0.0588
Epoch 6/20
1349/1349 [==============================] - 23s 17ms/step - loss: 0.0577 - mae: 0.0577 - val_loss: 0.0597 - val_mae: 0.0597
Epoch 7/20
1349/1349 [==============================] - 23s 17ms/step - loss: 0.0568 - mae: 0.0568 - val_loss: 0.0575 - val_mae: 0.0575
Epoch 8/20
1349/1349 [==========================

In [14]:
total_mae = 0
i = 100
count = 0
y_mean_list = []
y_pred_mean_list = []
for x, y in valid_set.take(i):
    y_pred = model(x)
    y_mean_list.append(y.numpy().mean())
    y_pred_mean_list.append(y_pred.numpy().mean())
    mae = np.mean(np.abs(y.numpy() - y_pred.numpy()))
    total_mae += mae
    count += 1
total_mae /= count
print(y_mean_list[:15])
print(y_pred_mean_list[:15])
print(total_mae)

[0.2860595, 0.25557086, 0.2915365, 0.33576047, 0.30199873, 0.24839891, 0.15883857, 0.14882076, 0.18442619, 0.22276969, 0.26605153, 0.16713813, 0.13072902, 0.122702315, 0.10736147]
[0.28207186, 0.2843352, 0.25927487, 0.31562564, 0.2935661, 0.29329696, 0.16311651, 0.1436135, 0.15680802, 0.16621979, 0.20332561, 0.17216176, 0.13226289, 0.1275591, 0.116526276]
0.05357529351487756


# pytorch

In [15]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, series, n_past, n_future):
        self.series = series
        self.n_past = n_past
        self.n_future = n_future

    def __len__(self):
        return self.series.shape[0] - self.n_past - self.n_future

    def __getitem__(self, idx):
        X = self.series[idx : idx+self.n_past, :]
        y = self.series[idx+self.n_past : idx+self.n_past+self.n_future, :]
        return X, y

n_past = 24
n_future = 24
timeSeries_DS = TimeSeriesDataset(train_data, n_past, n_future)
x_torch, y_torch = timeSeries_DS[0]

In [16]:
timeSeries_loader = DataLoader(timeSeries_DS, batch_size=32, shuffle=True)
x_torch,y_torch = next(iter(timeSeries_loader))
x_torch.shape, y_torch.shape

(torch.Size([32, 24, 7]), torch.Size([32, 24, 7]))

In [17]:
# causal padding을 지원하는 conv1d 층 구현
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, **kwargs)
        self.kernel_size = kernel_size

    def forward(self, x):
        # kernel_size - 1 만큼 패딩을 오른쪽(미래 데이터 측)에 추가
        # input : (batch, channel, sequence)
        padding = (0,self.kernel_size - 1,0,0)
        # pad has the form (padding_left,padding_right,padding_top,padding_bottom)
        x = F.pad(x, pad=padding, mode='constant')  # 'constant'는 0으로 패딩한다는 의미
        return self.conv(x)

In [18]:
# 참고
F.pad(x_torch, pad=(0,0,0,4), mode='constant').shape

torch.Size([32, 28, 7])

In [19]:
# 참고
print(x_torch.shape)
print(x_torch.permute(0,2,1).shape)
temp = CausalConv1d(in_channels=7, out_channels=32, kernel_size=3)
x_conv_torch = temp(x_torch.permute(0,2,1).float())
print(x_conv_torch.shape)

torch.Size([32, 24, 7])
torch.Size([32, 7, 24])
torch.Size([32, 32, 24])


In [21]:
# 참고
# Note: batch_first=True makes the input and output tensors of shape (batch, seq, feature)
print(x_conv_torch.permute(0,2,1).shape)
x_lstm1_torch, _ = nn.LSTM(input_size=32, hidden_size=64, batch_first=True, bidirectional=True)(x_conv_torch.permute(0,2,1))
print(x_lstm1_torch.shape)
x_lstm2_torch, _ = nn.LSTM(input_size=64, hidden_size=64, batch_first=True, bidirectional=True)(x_lstm1_torch)
print(x_lstm2_torch.shape)
x_dense1_torch = nn.Linear(in_features=64*2, out_features=32)(x_lstm2_torch)
print(x_dense1_torch.shape)

torch.Size([32, 24, 32])
torch.Size([32, 24, 128])
torch.Size([32, 24, 128])
torch.Size([32, 24, 32])


In [29]:
from IPython.core.debugger import set_trace

In [51]:
class timeSeriesModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1d = CausalConv1d(in_channels=7, out_channels=32, kernel_size=3)

        # Note: batch_first=True makes the input and output tensors of shape (batch, seq, feature)
        self.lstm1 = nn.LSTM(input_size=32, hidden_size=64, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=64*2, hidden_size=64, batch_first=True, bidirectional=True)
        self.dense1 = nn.Linear(in_features=64*2, out_features=32)
        self.dense2 = nn.Linear(in_features=32, out_features=16)
        self.dense3 = nn.Linear(in_features=16, out_features=7)

    def forward(self, x):
        if x.dim()==2:
            # channel
            x = x.unsqueeze(1)
        x = x.permute(0,2,1)
        x = torch.relu(self.conv1d(x.float()))
        x = x.permute(0, 2, 1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = torch.relu(self.dense1(x))
        x = torch.relu(self.dense2(x))
        x = self.dense3(x)

        return x


In [52]:
timeSeries_model = timeSeriesModel()
x_final = timeSeries_model(x_torch)
x_final.shape

torch.Size([32, 24, 7])

In [36]:
timeSeries_model = timeSeriesModel()
x_final = timeSeries_model(x_torch)
x_final.shape

torch.Size([32, 24, 7])

In [53]:
train_DS = TimeSeriesDataset(train_data, n_past, n_future)
train_loader = DataLoader(train_DS, batch_size=32, shuffle=True)
val_DS = TimeSeriesDataset(valid_data, n_past, n_future)
val_loader = DataLoader(val_DS, batch_size=32, shuffle=True)

In [54]:
timeSeries_model = timeSeriesModel()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
timeSeries_model.to(device)
optimizer = torch.optim.Adam(timeSeries_model.parameters(), lr=0.0005)
criterion = nn.L1Loss().to(device)

In [55]:
epochs = 40
for epoch in range(epochs):
    train_loss = 0
    train_mae = 0
    timeSeries_model.train()
    for inputs, targets in train_loader:
        inputs = inputs.float().to(device)
        targets = targets.float().to(device)
        outputs = timeSeries_model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss +=  loss.item() * inputs.size(0)
        train_mae += torch.mean(torch.abs(targets - outputs)) * inputs.size(0)
    train_loss /= len(train_DS)
    train_mae /= len(train_DS)


    val_loss = 0
    val_mae = 0
    timeSeries_model.eval()
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.float().to(device)
            targets = targets.float().to(device)
            outputs = timeSeries_model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            val_mae += torch.mean(torch.abs(targets - outputs)) * inputs.size(0)
    val_loss /= len(val_DS)
    val_mae /= len(val_DS)

    print(f'''Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Train MAE: {train_mae:.6f}, Val Loss: {val_loss:.6f}, Val MAE: {val_mae:.6f}''')

Epoch 1/40, Train Loss: 0.068534, Train MAE: 0.068534, Val Loss: 0.053746, Val MAE: 0.053746
Epoch 2/40, Train Loss: 0.051068, Train MAE: 0.051068, Val Loss: 0.051627, Val MAE: 0.051627
Epoch 3/40, Train Loss: 0.049924, Train MAE: 0.049924, Val Loss: 0.050810, Val MAE: 0.050810
Epoch 4/40, Train Loss: 0.050078, Train MAE: 0.050078, Val Loss: 0.049938, Val MAE: 0.049938
Epoch 5/40, Train Loss: 0.050079, Train MAE: 0.050079, Val Loss: 0.050146, Val MAE: 0.050145
Epoch 6/40, Train Loss: 0.048486, Train MAE: 0.048486, Val Loss: 0.049966, Val MAE: 0.049966
Epoch 7/40, Train Loss: 0.048086, Train MAE: 0.048086, Val Loss: 0.049805, Val MAE: 0.049805
Epoch 8/40, Train Loss: 0.047039, Train MAE: 0.047039, Val Loss: 0.049179, Val MAE: 0.049179
Epoch 9/40, Train Loss: 0.045717, Train MAE: 0.045717, Val Loss: 0.047152, Val MAE: 0.047152
Epoch 10/40, Train Loss: 0.044981, Train MAE: 0.044981, Val Loss: 0.047378, Val MAE: 0.047378
Epoch 11/40, Train Loss: 0.044450, Train MAE: 0.044450, Val Loss: 0.0

In [56]:
total_mae = 0
counts = 0
target_mean_list = []
output_mean_list = []
for inputs, targets in val_loader:
    inputs = inputs.float().to(device)
    targets = targets.float().to(device)
    outputs = timeSeries_model(inputs).squeeze()
    target_mean_list.append(targets.detach().cpu().numpy().mean())
    output_mean_list.append(outputs.detach().cpu().numpy().mean())
    mae = np.mean(np.abs(targets.cpu().detach().numpy() - outputs.detach().cpu().numpy()))
    total_mae += mae
    counts += 1
total_mae /= counts
print(target_mean_list[:15])
print(output_mean_list[:15])
print(total_mae)

[0.19166344, 0.19137448, 0.20841342, 0.17906198, 0.21587715, 0.2104038, 0.20672266, 0.19373319, 0.23610584, 0.19015433, 0.20844854, 0.21148664, 0.20723788, 0.21822807, 0.22518626]
[0.19274715, 0.18033658, 0.21612795, 0.18819961, 0.20976031, 0.19615929, 0.20551637, 0.19172361, 0.215268, 0.19013861, 0.19882125, 0.20873438, 0.19238204, 0.2173624, 0.19911987]
0.048282099157306684
